# Covermap Comparison

**Author:** Adam Yang (ayang115@umd.edu)

**Description:** Compares datasets against test sets

**Crop Maps Used:** 

In [2]:
import pandas as pd
import numpy as np
import geopandas as gdp
import os
import geemap
from sklearn.metrics import classification_report 

In [3]:
import ee

ee.Authenticate()

ee.Initialize()


Successfully saved authorization token.


## **Section 0** - Declaring Functions & Loading Test Data

In [4]:
# Remaps classes to crop/noncrop 
def map_values(val, value_for_crop):
    if val == value_for_crop:
        return 1
    else:
        return 0

In [5]:
# Function used in map function to extract from feature collection
def rasterExtraction(image, resolution, f_collection):
    feature = image.sampleRegions(
        collection = f_collection,
        scale = resolution
    )
    return feature

In [6]:
# Creates ee.Feature from longitude and latitude coordinates from a dataframe
def create_point(row):
    geom = ee.Geometry.Point(row["lon"], row["lat"])
    prop = dict(row)

    return ee.Feature(geom, prop)

In [7]:
test_files = ["Togo.csv", "Kenya.csv"]
dir = "../data/datasets/"

In [8]:
test_data = pd.DataFrame(columns=["lat", "lon", "test_class", "ee_pts", "country"])

test_set = []
for file in test_files:
    # Set dict key name
    key = file.split('.')[0]

    # Read in data and extract test values and points 
    df = pd.read_csv((dir+file))
    df = df.loc[df["subset"] == "testing"]
    df = df[["lat", "lon", "class_probability"]]

    # Create earth engine geometry points
    df["ee_pts"] = df.apply(create_point, axis=1)

    # Recast points as 1 or 0 (threshold = 0.5)
    df["test_class"] = df["class_probability"].apply(lambda x: 1 if x>=0.5 else 0)

    df["country"] = key

    test_set.append(df)

test_data = pd.concat(test_set)

In [19]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1184 entries, 1276 to 8657
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lat                1184 non-null   float64
 1   lon                1184 non-null   float64
 2   class_probability  1184 non-null   float64
 3   ee_pts             1184 non-null   object 
 4   test_class         1184 non-null   int64  
 5   country            1184 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 64.8+ KB


In [10]:
test_coll = ee.FeatureCollection(test_data["ee_pts"].tolist())

## **Section 1** - Harvest Data

## **Section 2** - Earth Engine Accessible

**Copernicus Land Cover** ([Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#description))

In [16]:
# Load copernicus data
copernicus = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
cop_results = copernicus.select("discrete_classification").filterDate("2019-01-01", "2020-01-01").map(lambda x: rasterExtraction(x, 100, test_coll)).flatten()

In [24]:
cop_sampled = geemap.ee_to_gdf(cop_results)
cop_sampled["cop_class"] = cop_sampled["discrete_classification"].apply(lambda x: map_values(x, 40))
cop_sampled.head()

,geometry,class_probability,discrete_classification,lat,lon,cop_class
0,None,0.00,20,9.875907,1.172471,0
1,None,0.25,124,9.180105,1.374695,0
2,None,0.00,116,9.275314,1.196385,0
3,None,0.00,114,7.346575,0.705167,0
4,None,0.25,112,6.777337,0.629817,0


In [25]:
cop_sampled = pd.merge(test_data[["lat","lon","test_class", "country"]], cop_sampled, on=["lat", "lon"])
cop_sampled.head()

,lat,lon,test_class,country,geometry,class_probability,discrete_classification,cop_class
0,9.875907,1.172471,0,Togo,None,0.00,20,0
1,9.180105,1.374695,0,Togo,None,0.25,124,0
2,9.275314,1.196385,0,Togo,None,0.00,116,0
3,7.346575,0.705167,0,Togo,None,0.00,114,0
4,6.777337,0.629817,0,Togo,None,0.25,112,0


In [27]:
cop_reports = {}

for country, df in cop_sampled.groupby("country"):
    cop_reports[country] = pd.DataFrame.from_dict(
    classification_report(cop_sampled["test_class"], cop_sampled["cop_class"], output_dict=True)
    )

cop_reports["Kenya"]

,0,1,accuracy,macro avg,weighted avg
precision,0.782696,0.895197,0.847973,0.838946,0.851394
recall,0.843818,0.850622,0.847973,0.847220,0.847973
f1-score,0.812109,0.872340,0.847973,0.842224,0.848889
support,461.000000,723.000000,0.847973,1184.000000,1184.000000


**ESA World Cover** ([Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100)) 

In [51]:
esa = ee.ImageCollection("ESA/WorldCover/v100")
esa_results = esa.filterBounds(test_coll).map(lambda x: rasterExtraction(x, 10, test_coll)).flatten()

In [58]:
esa_sampled = geemap.ee_to_gdf(esa_results)
esa_sampled["esa_class"] = esa_sampled["Map"].apply(lambda x: map_values(x, 40))
esa_sampled.head()

,geometry,Map,class_probability,lat,lon,esa_class
0,None,20,0.00,9.875907,1.172471,0
1,None,20,0.25,9.180105,1.374695,0
2,None,10,0.00,9.275314,1.196385,0
3,None,10,0.00,7.346575,0.705167,0
4,None,10,0.25,6.777337,0.629817,0


In [59]:
esa_sampled = pd.merge(test_data[["lat","lon","test_class", "country"]], esa_sampled, on=["lat", "lon"])
esa_sampled.head()

,lat,lon,test_class,country,geometry,Map,class_probability,esa_class
0,9.875907,1.172471,0,Togo,None,20,0.00,0
1,9.180105,1.374695,0,Togo,None,20,0.25,0
2,9.275314,1.196385,0,Togo,None,10,0.00,0
3,7.346575,0.705167,0,Togo,None,10,0.00,0
4,6.777337,0.629817,0,Togo,None,10,0.25,0


In [67]:
esa_reports = {}

for country, df in esa_sampled.groupby("country"):
    esa_reports[country] = pd.DataFrame.from_dict(
    classification_report(esa_sampled["test_class"], esa_sampled["esa_class"], output_dict=True)
    )

esa_reports["Kenya"]

,0,1,accuracy,macro avg,weighted avg
precision,0.455657,0.931034,0.537162,0.693346,0.745943
recall,0.969631,0.261411,0.537162,0.615521,0.537162
f1-score,0.619972,0.408207,0.537162,0.514090,0.490660
support,461.000000,723.000000,0.537162,1184.000000,1184.000000


**GLAD Global** ([Earth Engine](https://glad.earthengine.app/view/global-cropland-dynamics))

In [30]:
glad = ee.ImageCollection("users/potapovpeter/Global_cropland_2019")
glad_results = glad.filterBounds(test_coll).map(lambda x: rasterExtraction(x, 30, test_coll)).flatten()

In [31]:
glad_sampled = geemap.ee_to_gdf(glad_results)
glad_sampled.head()

,geometry,b1,class_probability,lat,lon
0,None,0,0.00,9.875907,1.172471
1,None,0,0.25,9.180105,1.374695
2,None,0,0.00,9.275314,1.196385
3,None,0,0.00,7.346575,0.705167
4,None,0,0.25,6.777337,0.629817


In [69]:
glad_sampled = pd.merge(test_data[["lat","lon","test_class", "country"]], glad_sampled, on=["lat", "lon"])

In [71]:
glad_reports = {}

for country, df in esa_sampled.groupby("country"):
    glad_reports[country] = pd.DataFrame.from_dict(
        classification_report(glad_sampled["test_class"], glad_sampled["b1"], output_dict=True)
        )

glad_reports["Kenya"]

,0,1,accuracy,macro avg,weighted avg
precision,0.700692,0.907591,0.806588,0.804141,0.827033
recall,0.878525,0.760719,0.806588,0.819622,0.806588
f1-score,0.779596,0.827690,0.806588,0.803643,0.808964
support,461.000000,723.000000,0.806588,1184.000000,1184.000000
